In [67]:
"""
Function: Ingest Smart Motorways highways (Filetype?) files from model shapefiles and process for DDB upload.
"""
import xlsxwriter
import pandas as pd
import pandas.io.formats.excel
import json
import os

In [68]:
reference_marker_map = {
    "DESIGN_ID": "Model design id",
    "DATE_INSTA": "Date of installation",
    "DEP_DAS_ID": "Departure - DAS ID",
    "EXPECTED_S": "Expected service life",
    "ASSET_INFO": "Additional information",
    "ASSET_ID": "Access code",
    "AST_LOCTXT": "Location text",
    "COLOUR": "Colour",
    "ENDCHAIN": "End chainage",
    "END_DATE": "End date",
    "MANUFACTUR": "Manufacturer",
    "OWNER": "Owner of inventory item",
    "PROD_NAME": "Product name/id",
    "SECTION": "Section",
    "STCHAIN": "Start chainage",
    "START_DATE": "Start date",
    "XCOORD": "Easting",
    "XSP": "Cross section position",
    "YCOORD": "Northing",
    "CURRENT_MA": "Current maintenance contract",
    "CURRENTLY_": "Currently maintained by",
    "OSOURCE_ID": "Supplier external reference id",
    "SYSTEM_ID": "System id"
}

utility_marker_map = {
    "DESIGN_ID": "Model design id",
    "DATE_INSTA": "Date of installation",
    "DEP_DAS_ID": "Departure - DAS ID",
    "EXPECTED_S": "Expected service life",
    "ASSET_INFO": "Additional information",
    "AST_LOCTXT": "Location text",
    "COLOUR": "Colour",
    "ENDCHAIN": "End chainage",
    "END_DATE": "End date",
    "MANUFACTUR": "Manufacturer",
    "MATERIAL": "Material",
    "OWNER": "Owner of inventory item",
    "PROD_NAME": "Product name/id",
    "SECTION": "Section",
    "STCHAIN": "Start chainage",
    "START_DATE": "Start date",
    "TYPE": "Type",
    "XCOORD": "Easting",
    "XSP": "Cross section position",
    "YCOORD": "Northing",
    "CURRENT_MA": "Current maintenance contract",
    "CURRENTLY_": "Currently maintained by",
    "OSOURCE_ID": "Supplier external reference id",
    "SYSTEM_ID": "System id"
}

bollard_map = {
    "DESIGN_ID": "Model design id",
    "DATE_INSTA": "Date of installation",
    "DEP_DAS_ID": "Departure - DAS ID",
    "EXPECTED_S": "Expected service life",
    "ASSET_INFO": "Additional information",
    "ASSET_ID": "Access code",
    "AST_LOCTXT": "Location text",
    "ENDCHAIN": "End chainage",
    "END_DATE": "End date",
    "ILLUMINATE": "Illuminated",
    "MANUFACTUR": "Manufacturer",
    "MATERIAL": "Material",
    "OWNER": "Owner of inventory item",
    "PASSIVELY_": "Passively safe",
    "PROD_NAME": "Product name/id",
    "SECTION": "Section",
    "STCHAIN": "Start chainage",
    "START_DATE": "Start date",
    "TSRGD_DIAG": "TSRGD diagram number",
    "TYPE": "Type",
    "XCOORD": "Easting",
    "XSP": "Cross section position",
    "YCOORD": "Northing",
    "CURRENT_MA": "Current maintenance contract",
    "CURRENTLY_": "Currently maintained by",
    "OSOURCE_ID": "Supplier external reference id",
    "SYSTEM_ID": "System id"
}

In [69]:
print(len(reference_marker_map))
print(len(utility_marker_map))
print(len(bollard_map))

23
24
27


In [70]:
# input files
input_files = os.fsencode("..\\files\\input\\schedules\\")
df_dict = {}

for file in os.listdir(input_files):
    filename = os.fsdecode(file)
    df_dict[filename] = pd.DataFrame()
   
    workbook_df = pd.read_excel("..\\files\\input\\schedules\\"+filename, sheet_name=None, engine="openpyxl")

    for sheet_name, sheet_df in workbook_df.items():
        print(sheet_name)
        df_dict[sheet_name] = sheet_df

Reference Marker
Utility Marker Post
RRSB - Bollard


In [71]:
print(df_dict["Utility Marker Post"].to_string())

    DESIGN_ID DATE_INSTA  DEP_DAS_ID  EXPECTED_S  ASSET_INFO  AST_LOCTXT  COLOUR  ENDCHAIN  END_DATE          MANUFACTUR MATERIAL  OWNER                       PROD_NAME  SECTION  STCHAIN  START_DATE            TYPE       XCOORD  XSP       YCOORD  CURRENT_MA  CURRENTLY_  OSOURCE_ID  SYSTEM_ID
0         NB3 2020-11-19         NaN         NaN         NaN         NaN  YELLOW       NaN       NaN  GLASDON FLEXMASTER    OTHER    NaN             UTILITY MARKER POST      NaN      NaN         NaN  OVERHEAD POWER  488141.9020  NaN  241543.1630         NaN         NaN         NaN        NaN
1         NB4 2021-05-11         NaN         NaN         NaN         NaN  YELLOW       NaN       NaN  GLASDON FLEXMASTER    OTHER    NaN             UTILITY MARKER POST      NaN      NaN         NaN  OVERHEAD POWER  488120.6960  NaN  241558.4180         NaN         NaN         NaN        NaN
2         NB5 2021-05-11         NaN         NaN         NaN         NaN  YELLOW       NaN       NaN  GLASDON FLEXMASTER 

In [72]:
# ingest road marking model exports and combine rows that share the same asset ID and therefore have multiple element IDs

input_files = os.fsencode("..\\files\\input\\exports\\")
exports_df = pd.DataFrame()
for file in os.listdir(input_files):
    filename = os.fsdecode(file)
    exports_df = pd.concat([exports_df, pd.read_csv("..\\files\\input\\exports\\" + filename)])
         
new_df = exports_df.copy()
new_df = new_df.iloc[0:0]

asset_ids = exports_df["SUPP_REFF"].to_list()
for i, line in exports_df.iterrows():
    if line["SUPP_REFF"] in new_df["SUPP_REFF"].to_list():
        continue

    dups = exports_df.loc[exports_df["SUPP_REFF"] == line["SUPP_REFF"]]

    if len(dups["Asset ID"].to_list()) == 1:
        element_ids = dups["Asset ID"].to_list()[0]
    if len(dups["Asset ID"].to_list()) > 1:
        element_ids = ", ".join(map(str, dups["Asset ID"].to_list()))

    new_row = []
    new_row.append(line["SUPP_REFF"])
    new_row.append(line["Layer"])
    new_row.append(element_ids)
    new_row.append(line["Drawing Name"])
    new_row.append(line["Area Code"])
    new_row.append(line["Direction"])

    new_df.loc[len(new_df)] = new_row


In [73]:
print(new_df.to_string())

     SUPP_REFF                                   Layer        Asset ID                             Drawing Name Area Code Direction
0      MPNB-34                     C-ASSEMBLIES_Points          654173   HA549348-AMAR-HSN-J13-M3-CH-000002.dwg       J13        MP
1     MPSB-343                     C-ASSEMBLIES_Points          654176   HA549348-AMAR-HSN-J13-M3-CH-000002.dwg       J13        MP
2      MPNB-33                     C-ASSEMBLIES_Points          654179   HA549348-AMAR-HSN-J13-M3-CH-000002.dwg       J13        MP
3      MPNB-32                     C-ASSEMBLIES_Points          654182   HA549348-AMAR-HSN-J13-M3-CH-000002.dwg       J13        MP
4      MPNB-31                     C-ASSEMBLIES_Points          654185   HA549348-AMAR-HSN-J13-M3-CH-000002.dwg       J13        MP
5      MPNB-30                     C-ASSEMBLIES_Points          654188   HA549348-AMAR-HSN-J13-M3-CH-000002.dwg       J13        MP
6      MPNB-29                     C-ASSEMBLIES_Points          654191   HA5

In [74]:
# merge schedules and exports
sheet_names = ["Reference Marker", "Utility Marker Post", "RRSB - Bollard"]

for sh in sheet_names:
    df_dict[sh] = df_dict[sh].merge(new_df, how="left", left_on="DESIGN_ID", right_on="SUPP_REFF")
    # Drop Assets which haven't been modelled yet
    df_dict[sh] = df_dict[sh].drop(df_dict[sh].loc[df_dict[sh]["SUPP_REFF"].isnull()].index)

# df_dict["Post (Signs)"]["Asset ID"] = df_dict["Post (Signs)"]["Asset ID"].astype(int)
# df_dict["Sign Face "]["Asset ID"] = df_dict["Sign Face "]["Asset ID"].astype(int)

In [75]:
print(df_dict["Reference Marker"].to_string())

    DESIGN_ID DATE_INSTA  DEP_DAS_ID  EXPECTED_S  ASSET_INFO ASSET_ID  AST_LOCTXT COLOUR  ENDCHAIN  END_DATE MANUFACTUR  OWNER             PROD_NAME  SECTION  STCHAIN  START_DATE      XCOORD  XSP      YCOORD  CURRENT_MA  CURRENTLY_  OSOURCE_ID  SYSTEM_ID SUPP_REFF                       Layer Asset ID                             Drawing Name Area Code Direction
0      MPNB-1        NaT         NaN         NaN         NaN     72/7         NaN   BLUE       NaN       NaN        NaN    NaN  DISTANCE MARKER POST      NaN      NaN         NaN  495988.028  NaN  237333.712         NaN         NaN         NaN        NaN    MPNB-1         C-ASSEMBLIES_Points   654276   HA549348-AMAR-HSN-J13-M3-CH-000002.dwg       J13        MP
1      MPNB-2        NaT         NaN         NaN         NaN     72/8         NaN   BLUE       NaN       NaN        NaN    NaN  DISTANCE MARKER POST      NaN      NaN         NaN  495889.383  NaN  237370.574         NaN         NaN         NaN        NaN    MPNB-2         C

In [76]:
component_map = {
    "Reference Marker": ["RMP", "RFM"],
    "Utility Marker Post": ["RMP", "UTM"],
    "RRSB - Bollard": ["RTS", "BOL"]
}

In [77]:
sheet_names = ["Reference Marker", "Utility Marker Post", "RRSB - Bollard"]

for sh in sheet_names:

    # create DDB Taxonomy IDs
    df_dict[sh]["Arup reference ID"] = ""
    delete_ls = []
    asset_numbering = {}

    for i, line in df_dict[sh].iterrows():
        
        # Set variables to each level of the taxonomy ID
        network_link = "IT_SM:IT_SM-M:M1"
        if line['Area Code'][:2] == "ML":
            section = "LK" + line['Area Code'][2:]
        elif line['Area Code'] == "NPS":
            section = "SER"
        else:
            section = line['Area Code']

        if "NB" in line["SUPP_REFF"]:
            direction = "NB"
        elif "SB" in line["SUPP_REFF"]:
            direction = "SB"
        else:
            direction = "ND"

        componentgroupcode = component_map[sh][0]
        componentsubgroupcode = component_map[sh][1]

        asset_index = 1

        if section in asset_numbering:
            if direction in asset_numbering[section]:
                if componentgroupcode in asset_numbering[section][direction]:
                    if componentsubgroupcode in asset_numbering[section][direction][componentgroupcode]:
                        asset_numbering[section][direction][componentgroupcode][componentsubgroupcode] += 1
                        asset_index = asset_numbering[section][direction][componentgroupcode][componentsubgroupcode]
                    else:
                        asset_numbering[section][direction][componentgroupcode][componentsubgroupcode] = asset_index
                        asset_index = asset_numbering[section][direction][componentgroupcode][componentsubgroupcode]
                else:
                    asset_numbering[section][direction][componentgroupcode] = {}
                    asset_numbering[section][direction][componentgroupcode][componentsubgroupcode] = asset_index
                    asset_index = asset_numbering[section][direction][componentgroupcode][componentsubgroupcode]
            else:
                asset_numbering[section][direction] = {}
                asset_numbering[section][direction][componentgroupcode] = {}
                asset_numbering[section][direction][componentgroupcode][componentsubgroupcode] = asset_index
                asset_index = asset_numbering[section][direction][componentgroupcode][componentsubgroupcode]
        else:
            asset_numbering[section] = {}
            asset_numbering[section][direction] = {}
            asset_numbering[section][direction][componentgroupcode] = {}
            asset_numbering[section][direction][componentgroupcode][componentsubgroupcode] = asset_index
            asset_index = asset_numbering[section][direction][componentgroupcode][componentsubgroupcode]
        
        taxonomy_id = network_link+"-"+section+"-"+direction+"-"+componentgroupcode+"-"+componentsubgroupcode+"-"+componentsubgroupcode+str(asset_index).zfill(2)
        print(taxonomy_id)
        df_dict[sh]["Arup reference ID"][i] = taxonomy_id

IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM01
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM02
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM03
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM04
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM05
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM06
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM07
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM08
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM09
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM10
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM11
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM12
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM13
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM14
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM15
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM16
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM17
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM18
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM19
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM20
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM21
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM22
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM23
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM24
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM25
IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM26
IT_SM:IT_SM-

C:\Users\Jamie.Butler\AppData\Local\Temp\ipykernel_54972\705241348.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dict[sh]["Arup reference ID"][i] = taxonomy_id


IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM86
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM87
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM88
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM89
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM90
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM91
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM92
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM93
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM94
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM95
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM96
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM97
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM98
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM99
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM100
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM101
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM102
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM103
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM104
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM105
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM106
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM107
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM108
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM109
IT_SM:IT_SM-M:M1-LK02-SB-RMP-RFM-RFM110
IT_SM:IT_SM-M:

C:\Users\Jamie.Butler\AppData\Local\Temp\ipykernel_54972\705241348.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dict[sh]["Arup reference ID"][i] = taxonomy_id
C:\Users\Jamie.Butler\AppData\Local\Temp\ipykernel_54972\705241348.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dict[sh]["Arup reference ID"][i] = taxonomy_id


In [78]:
print(df_dict["Reference Marker"].to_string())

    DESIGN_ID DATE_INSTA  DEP_DAS_ID  EXPECTED_S  ASSET_INFO ASSET_ID  AST_LOCTXT COLOUR  ENDCHAIN  END_DATE MANUFACTUR  OWNER             PROD_NAME  SECTION  STCHAIN  START_DATE      XCOORD  XSP      YCOORD  CURRENT_MA  CURRENTLY_  OSOURCE_ID  SYSTEM_ID SUPP_REFF                       Layer Asset ID                             Drawing Name Area Code Direction                        Arup reference ID
0      MPNB-1        NaT         NaN         NaN         NaN     72/7         NaN   BLUE       NaN       NaN        NaN    NaN  DISTANCE MARKER POST      NaN      NaN         NaN  495988.028  NaN  237333.712         NaN         NaN         NaN        NaN    MPNB-1         C-ASSEMBLIES_Points   654276   HA549348-AMAR-HSN-J13-M3-CH-000002.dwg       J13        MP    IT_SM:IT_SM-M:M1-J13-NB-RMP-RFM-RFM01
1      MPNB-2        NaT         NaN         NaN         NaN     72/8         NaN   BLUE       NaN       NaN        NaN    NaN  DISTANCE MARKER POST      NaN      NaN         NaN  495889.383  

In [79]:
print(len(df_dict["Reference Marker"]))
print(len(df_dict["Utility Marker Post"]))
print(len(df_dict["RRSB - Bollard"]))

749
47
129


In [ ]:
sheet_names = ["Reference Marker", "Utility Marker Post", "RRSB - Bollard"]

for sh in sheet_names:

    data = df_dict[sh]

    data = data.drop(columns=["Layer", "Drawing Name", "Direction", "Area Code", "SUPP_REFF"])

    if sh == "Reference Marker":
        data = data.rename(columns=reference_marker_map)
    elif sh == "Utility Marker Post":
        data = data.rename(columns=utility_marker_map)
    elif sh == "RRSB - Bollard":
        data = data.rename(columns=bollard_map)

    data.to_csv("../files/output/"+ sh + ".csv", index=False)

In [82]:
# input files
input_files = os.fsencode("..\\files\\input\\schedules\\")
df_dict2 = {}

for file in os.listdir(input_files):
    filename = os.fsdecode(file)
    df_dict2[filename] = pd.DataFrame()
   
    workbook_df = pd.read_excel("..\\files\\input\\schedules\\"+filename, sheet_name=None, engine="openpyxl")

    for sheet_name, sheet_df in workbook_df.items():
        df_dict2[sheet_name] = sheet_df

In [88]:
ids = list(df_dict["RRSB - Bollard"]["DESIGN_ID"])

for i, line in df_dict2["RRSB - Bollard"].iterrows():
    if line["DESIGN_ID"] not in ids:
        print(line["DESIGN_ID"]+ ", ")

HMPNB-16, 
HMPNB-17, 
HMPSB-42, 
HMPNB-49, 
HMPSB-18, 
HMPSB-66, 


In [85]:
print(new_df.to_string())

     SUPP_REFF                                   Layer        Asset ID                             Drawing Name Area Code Direction
0      MPNB-34                     C-ASSEMBLIES_Points          654173   HA549348-AMAR-HSN-J13-M3-CH-000002.dwg       J13        MP
1     MPSB-343                     C-ASSEMBLIES_Points          654176   HA549348-AMAR-HSN-J13-M3-CH-000002.dwg       J13        MP
2      MPNB-33                     C-ASSEMBLIES_Points          654179   HA549348-AMAR-HSN-J13-M3-CH-000002.dwg       J13        MP
3      MPNB-32                     C-ASSEMBLIES_Points          654182   HA549348-AMAR-HSN-J13-M3-CH-000002.dwg       J13        MP
4      MPNB-31                     C-ASSEMBLIES_Points          654185   HA549348-AMAR-HSN-J13-M3-CH-000002.dwg       J13        MP
5      MPNB-30                     C-ASSEMBLIES_Points          654188   HA549348-AMAR-HSN-J13-M3-CH-000002.dwg       J13        MP
6      MPNB-29                     C-ASSEMBLIES_Points          654191   HA5